# PySpark Tutorial - Data Transformation

Create the Spark Session required for any PySpark program.  Most programs will store this in a variable named `spark`.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, concat, to_date, rank
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, DateType, StringType
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("PySparkTutorial").getOrCreate()

## Read the Source Data
The following code creates a DataFrame for both the policies and claims by reading CSV files.

In [2]:
policyDF = spark.read.option("header",True).csv("./data/policy.csv") \
                .withColumn("sum_insured", col("sum_insured").cast(IntegerType())) \
                .withColumn("vehicle_age", col("vehicle_age").cast(IntegerType())) \
                .withColumn("premium", col("premium").cast(IntegerType()))
claimsDF = spark.read.option("header",True).csv("./data/claims.csv") \
                .withColumn("cost", col("cost").cast(IntegerType()))

Analysis of the Schemas show that the date variables are `string` types rather than Spark `date` types.

In [3]:
policyDF.printSchema()
claimsDF.printSchema()

root
 |-- policy: string (nullable = true)
 |-- make: string (nullable = true)
 |-- vehicle_age: integer (nullable = true)
 |-- sum_insured: integer (nullable = true)
 |-- inception_date: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- premium: integer (nullable = true)

root
 |-- policy: string (nullable = true)
 |-- incident_date: string (nullable = true)
 |-- cost: integer (nullable = true)



## Creating new Columns
The `withColumn()` method can be used to create new column in a number of ways.

`when()` and `otherwise` can be used to create columns using similar logic to SQL `case` statements.

Note that referring to columns in code can be done using `dataframe.column-name` or `col("column-name")` in this case with equal results.

In [4]:
# These two methods of creating policyDF produce identical results.
policyDF = policyDF.withColumn("status", when(policyDF.start_date==policyDF.inception_date, "New Business") \
                                        .otherwise("Renewal"))
policyDF = policyDF.withColumn("status", when(col("start_date")==col("inception_date"), "New Business") \
                                        .otherwise("Renewal"))

The `select()` method can be used to both subset the number of columns, and also order them in the resulting DataFrame.

In [5]:
policyDF.select(["policy", "start_date", "status", "make", "sum_insured"]).show(5)

+-------+----------+------------+------+-----------+
| policy|start_date|      status|  make|sum_insured|
+-------+----------+------------+------+-----------+
|CAR0001|  20180101|New Business|TOYOTA|      15000|
|CAR0001|  20190101|     Renewal|TOYOTA|      13500|
|CAR0001|  20200101|     Renewal|TOYOTA|      12000|
|CAR0002|  20200210|New Business|SUBARU|      14000|
|CAR0003|  20180315|New Business|  FORD|      10000|
+-------+----------+------------+------+-----------+
only showing top 5 rows



## Transformation using a Python Function
A python function can provide abilities that are not available in SQL.

The following function will iterate over all columns in a DataFrame using `df.columns`. This appears to be constantly creating new DataFrames, however it should be remembered that Spark uses Lazy Evaluation, and this is just creating a chain of instructions that can later be optimised.

In [6]:
def fix_dates(df):
    """Find all columns named *_date and convert from string to Spark Date type."""
    for col in df.columns:
        if col.endswith("_date") and dict(df.dtypes)[col]=='string':
            print("NOTE: Fixing date column '{}'.".format(col))
            df = df.withColumn(col, to_date(df[col], "yyyyMMdd"))
    return df

policyDF = fix_dates(policyDF)
claimsDF = fix_dates(claimsDF)

NOTE: Fixing date column 'inception_date'.
NOTE: Fixing date column 'start_date'.
NOTE: Fixing date column 'end_date'.
NOTE: Fixing date column 'incident_date'.


Viewing the updated DataFrames shows that the date columns are now using the Spark `date` types.

In [7]:
policyDF.printSchema()
policyDF.show(5)

root
 |-- policy: string (nullable = true)
 |-- make: string (nullable = true)
 |-- vehicle_age: integer (nullable = true)
 |-- sum_insured: integer (nullable = true)
 |-- inception_date: date (nullable = true)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)
 |-- premium: integer (nullable = true)
 |-- status: string (nullable = false)

+-------+------+-----------+-----------+--------------+----------+----------+-------+------------+
| policy|  make|vehicle_age|sum_insured|inception_date|start_date|  end_date|premium|      status|
+-------+------+-----------+-----------+--------------+----------+----------+-------+------------+
|CAR0001|TOYOTA|          1|      15000|    2018-01-01|2018-01-01|2018-12-31|   1000|New Business|
|CAR0001|TOYOTA|          2|      13500|    2018-01-01|2019-01-01|2019-12-31|    900|     Renewal|
|CAR0001|TOYOTA|          3|      12000|    2018-01-01|2020-01-01|2020-12-31|    800|     Renewal|
|CAR0002|SUBARU|          2|      140

## Joining DataFrames

In [8]:
combinedDF = policyDF.select(["policy", "make", "vehicle_age", "sum_insured",
                              "start_date", "end_date", "premium"]) \
                     .join(claimsDF, (policyDF.policy==claimsDF.policy) &
                                     (policyDF.start_date<=claimsDF.incident_date) &
                                     (policyDF.end_date>=claimsDF.incident_date))
combinedDF.show()

+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+
| policy|  make|vehicle_age|sum_insured|start_date|  end_date|premium| policy|incident_date| cost|
+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+
|CAR0001|TOYOTA|          3|      12000|2020-01-01|2020-12-31|    800|CAR0001|   2020-06-05| 5000|
|CAR0004| MAZDA|          5|      10000|2020-04-02|2021-04-01|    700|CAR0004|   2020-06-10| 3000|
|CAR0007|   BMW|          4|      24000|2020-07-13|2021-07-12|   1600|CAR0007|   2020-09-10|24000|
|CAR0009| TESLA|          1|      72000|2019-09-17|2020-09-16|   4800|CAR0009|   2020-02-10|15000|
+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+



The duplicate `policy` column shown above can be easily removed with a `drop()`.

In [9]:
combinedDF = policyDF.select(["policy", "make", "vehicle_age", "sum_insured",
                              "start_date", "end_date", "premium"]) \
                     .join(claimsDF, (policyDF.policy==claimsDF.policy) &
                                     (policyDF.start_date<=claimsDF.incident_date) &
                                     (policyDF.end_date>=claimsDF.incident_date)) \
                     .drop(claimsDF.policy)
combinedDF.show(5)

+-------+------+-----------+-----------+----------+----------+-------+-------------+-----+
| policy|  make|vehicle_age|sum_insured|start_date|  end_date|premium|incident_date| cost|
+-------+------+-----------+-----------+----------+----------+-------+-------------+-----+
|CAR0001|TOYOTA|          3|      12000|2020-01-01|2020-12-31|    800|   2020-06-05| 5000|
|CAR0004| MAZDA|          5|      10000|2020-04-02|2021-04-01|    700|   2020-06-10| 3000|
|CAR0007|   BMW|          4|      24000|2020-07-13|2021-07-12|   1600|   2020-09-10|24000|
|CAR0009| TESLA|          1|      72000|2019-09-17|2020-09-16|   4800|   2020-02-10|15000|
+-------+------+-----------+-----------+----------+----------+-------+-------------+-----+



Note that multiple `select`s can appear in the DataFrame logic as required.  The optimiser should help however forcing an early `select` will not hurt!

In [10]:
combinedDF = policyDF.select(["policy", "make", "vehicle_age", "sum_insured", "start_date", "end_date"]) \
                     .join(claimsDF, (policyDF.policy==claimsDF.policy) &
                                     (policyDF.start_date<=claimsDF.incident_date) &
                                     (policyDF.end_date>=claimsDF.incident_date)) \
                     .drop(claimsDF.policy) \
                     .select(["policy", "make", "vehicle_age", "sum_insured", "incident_date", "cost"])
combinedDF.show(5)

+-------+------+-----------+-----------+-------------+-----+
| policy|  make|vehicle_age|sum_insured|incident_date| cost|
+-------+------+-----------+-----------+-------------+-----+
|CAR0001|TOYOTA|          3|      12000|   2020-06-05| 5000|
|CAR0004| MAZDA|          5|      10000|   2020-06-10| 3000|
|CAR0007|   BMW|          4|      24000|   2020-09-10|24000|
|CAR0009| TESLA|          1|      72000|   2020-02-10|15000|
+-------+------+-----------+-----------+-------------+-----+



The default join type is `inner`, however this can be changed by providing the join style as the third parameter to the `.join()`.

In [11]:
combinedDF = policyDF.select(["policy", "make", "vehicle_age", "sum_insured", "start_date", "end_date"]) \
                     .join(claimsDF, (policyDF.policy==claimsDF.policy) &
                                     (policyDF.start_date<=claimsDF.incident_date) &
                                     (policyDF.end_date>=claimsDF.incident_date)
                                   , "left") \
                     .drop(claimsDF.policy) \
                     .select(["policy", "make", "vehicle_age", "sum_insured", "incident_date", "cost"]) \
                     .show()

+-------+-------+-----------+-----------+-------------+-----+
| policy|   make|vehicle_age|sum_insured|incident_date| cost|
+-------+-------+-----------+-----------+-------------+-----+
|CAR0001| TOYOTA|          1|      15000|         null| null|
|CAR0001| TOYOTA|          2|      13500|         null| null|
|CAR0001| TOYOTA|          3|      12000|   2020-06-05| 5000|
|CAR0002| SUBARU|          2|      14000|         null| null|
|CAR0003|   FORD|          6|      10000|         null| null|
|CAR0003|   FORD|          7|       9000|         null| null|
|CAR0003|   FORD|          8|       8000|         null| null|
|CAR0004|  MAZDA|          4|      11000|         null| null|
|CAR0004|  MAZDA|          5|      10000|   2020-06-10| 3000|
|CAR0005| HOLDEN|          9|       6000|         null| null|
|CAR0006| SUZUKI|          5|       4000|         null| null|
|CAR0007|    BMW|          4|      24000|   2020-09-10|24000|
|CAR0008|   AUDI|          3|      28000|         null| null|
|CAR0009

## Windowing Functions

Spark supports Windowing functions using similar syntax and logic to SQL queries.  Note that `sum()` is prefixed with the module alias `F` to prevent conflict with other functions of the same name.  This will appear frequently in sample code found on the internet.

In [12]:
policyDF = policyDF.withColumn("policy_term", rank().over(Window.partitionBy("policy").orderBy("start_date"))) \
                   .withColumn("total_premium", F.sum("premium").over(Window.partitionBy("policy")
                                                                            .orderBy("start_date"))) \
                   .withColumn("policy_id", concat(policyDF.policy, lit("~"), col("policy_term")))

policyDF.select(["policy", "start_date", "policy_term", "premium", "total_premium", "policy_id"]).show()

+-------+----------+-----------+-------+-------------+---------+
| policy|start_date|policy_term|premium|total_premium|policy_id|
+-------+----------+-----------+-------+-------------+---------+
|CAR0002|2020-02-10|          1|    950|          950|CAR0002~1|
|CAR0001|2018-01-01|          1|   1000|         1000|CAR0001~1|
|CAR0001|2019-01-01|          2|    900|         1900|CAR0001~2|
|CAR0001|2020-01-01|          3|    800|         2700|CAR0001~3|
|CAR0006|2020-06-18|          1|    300|          300|CAR0006~1|
|CAR0007|2020-07-13|          1|   1600|         1600|CAR0007~1|
|CAR0008|2020-08-11|          1|   1800|         1800|CAR0008~1|
|CAR0004|2019-04-02|          1|    750|          750|CAR0004~1|
|CAR0004|2020-04-02|          2|    700|         1450|CAR0004~2|
|CAR0009|2019-09-17|          1|   4800|         4800|CAR0009~1|
|CAR0009|2020-09-17|          2|   3500|         8300|CAR0009~2|
|CAR0010|2020-10-15|          1|    300|          300|CAR0010~1|
|CAR0005|2020-05-16|     

## Summarisation

The SQL concepts of `group by` and `order by` are also supported in PySpark.

The following example shows summarisation with `groupBy`:

In [13]:
summary = policyDF.groupBy("status").sum("sum_insured") \
                  .withColumnRenamed("sum(sum_insured)", "total_insured")
summary.show()

+------------+-------------+
|      status|total_insured|
+------------+-------------+
|     Renewal|       107500|
|New Business|       199000|
+------------+-------------+



Spark has the ability to perform more complex transformations including Cross Tabular DataFrames.

In [14]:
policyDF.crosstab("status", "make").show()

+------------+----+---+----+------+-------+-----+------+------+-----+------+
| status_make|AUDI|BMW|FORD|HOLDEN|HYUNDAI|MAZDA|SUBARU|SUZUKI|TESLA|TOYOTA|
+------------+----+---+----+------+-------+-----+------+------+-----+------+
|New Business|   1|  1|   1|     1|      1|    2|     1|     1|    1|     1|
|     Renewal|   0|  0|   2|     0|      0|    0|     0|     0|    1|     2|
+------------+----+---+----+------+-------+-----+------+------+-----+------+



The column names can be renamed in a number of ways including the `.alias` method and `.withColumnRenamed`.

In [15]:
ctabDF = policyDF.crosstab("status", "make")
ctabDF = ctabDF.select([col(x).alias(x.lower()) for x in ctabDF.columns]) \
               .withColumnRenamed("status_make", "status") \
               .show()

+------------+----+---+----+------+-------+-----+------+------+-----+------+
|      status|audi|bmw|ford|holden|hyundai|mazda|subaru|suzuki|tesla|toyota|
+------------+----+---+----+------+-------+-----+------+------+-----+------+
|New Business|   1|  1|   1|     1|      1|    2|     1|     1|    1|     1|
|     Renewal|   0|  0|   2|     0|      0|    0|     0|     0|    1|     2|
+------------+----+---+----+------+-------+-----+------+------+-----+------+



It helps to save resources if you `stop()` the Spark session when you are finished.  Note that by doing this you will be unable to re-run any of the code above without first re-creating the `spark` variable.

In [16]:
spark.stop()